In [1]:
from decouple import config
from multiprocessing import Process
from sqlalchemy import text
from time import sleep
#import boto3
import json
import random
import requests
import sqlalchemy
import yaml

random.seed(100)

In [2]:
class AWSDBConnector:

    def __init__(self):
        creds_config_access = config('credentials_env')
        with open(creds_config_access, 'r') as db_creds:
            self.creds= yaml.safe_load(db_creds)

    def create_db_connector(self):
        engine = sqlalchemy.create_engine(f"mysql+pymysql://{self.creds['USER']}:{self.creds['PASSWORD']}@{self.creds['HOST']}:{self.creds['PORT']}/{self.creds['DATABASE']}?charset=utf8mb4")
        return engine

new_connector = AWSDBConnector()

In [3]:
def run_infinite_post_data_loop():
    while True:
        sleep(random.randrange(0, 2))
        random_row = random.randint(0, 11000)
        engine = new_connector.create_db_connector()

        with engine.connect() as connection:

            pin_string = text(f"SELECT * FROM pinterest_data LIMIT {random_row}, 1")
            pin_selected_row = connection.execute(pin_string)
            
            for row in pin_selected_row:
                pin_result = dict(row._mapping)

            geo_string = text(f"SELECT * FROM geolocation_data LIMIT {random_row}, 1")
            geo_selected_row = connection.execute(geo_string)
            
            for row in geo_selected_row:
                geo_result = dict(row._mapping)

            user_string = text(f"SELECT * FROM user_data LIMIT {random_row}, 1")
            user_selected_row = connection.execute(user_string)
            
            for row in user_selected_row:
                user_result = dict(row._mapping) 

            print("**************************************************\n")
            print("pin result: ", pin_result, "\n")
            print("geo result: ", geo_result, "\n")
            print("user result: ", user_result, "\n")

            pin_payload = json.dumps({
            "records": [
                {
                    "value": {
                        "index": pin_result["index"],
                        "unique_id": pin_result["unique_id"],
                        "title": pin_result["title"],
                        "description": pin_result["description"],
                        "poster_name": pin_result["poster_name"],
                        "follower_count": pin_result["follower_count"],
                        "tag_list": pin_result["tag_list"],
                        "is_image_or_video": pin_result["is_image_or_video"],
                        "image_src": pin_result["image_src"],
                        "downloaded": pin_result["downloaded"],
                        "save_location": pin_result["save_location"],
                        "category": pin_result["category"]
                        }
                }
                ]}, default=str)
            headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
            invoke_url_pin = "https://k2lim6ri8g.execute-api.us-east-1.amazonaws.com/staging/topics/123d7d4a88ef.pin"
            pin_response = requests.request("POST", invoke_url_pin, headers=headers, data=pin_payload)
            print("pin status code: ", pin_response.status_code, "\n")

            geo_payload = json.dumps({
            "records": [
            {
                "value": {
                "geo_result":geo_result["ind"],
                "geo_result":geo_result["timestamp"],
                "geo_result":geo_result["latitude"],
                "geo_result":geo_result["longitude"],
                "geo_result":geo_result["country"],
                }
            }
            ]}, default=str)
            headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
            invoke_url_geo = "https://k2lim6ri8g.execute-api.us-east-1.amazonaws.com/staging/topics/123d7d4a88ef.geo"
            geo_response = requests.request("POST", invoke_url_geo, headers=headers, data=geo_payload)
            print("pin status code: ", geo_response.status_code, "\n")

            user_payload = json.dumps({
            "records": [
                {
                    "value": {
                        "user_result":user_result["ind"],
                        "user_result":user_result["first_name"],
                        "user_result":user_result["last_name"],
                        "user_result":user_result["age"],
                        "user_result":user_result["date_joined"],
                            }
                }
                ]}, default=str)
            headers = {'Content-Type': 'application/vnd.kafka.json.v2+json'}
            invoke_url_user = "https://k2lim6ri8g.execute-api.us-east-1.amazonaws.com/staging/topics/123d7d4a88ef.user"
            user_response = requests.request("POST", invoke_url_user, headers=headers, data=user_payload)
            print("pin status code: ", user_response.status_code, "\n")

In [4]:
run_infinite_post_data_loop()
print('working')

**************************************************

pin result:  {'index': 7528, 'unique_id': 'fbe53c66-3442-4773-b19e-d3ec6f54dddf', 'title': 'No Title Data Available', 'description': 'No description available Story format', 'poster_name': 'User Info Error', 'follower_count': 'User Info Error', 'tag_list': 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e', 'is_image_or_video': 'multi-video(story page format)', 'image_src': 'Image src error.', 'downloaded': 0, 'save_location': 'Local save in /data/mens-fashion', 'category': 'mens-fashion'} 

geo result:  {'ind': 7528, 'timestamp': datetime.datetime(2020, 8, 28, 3, 52, 47), 'latitude': -89.9787, 'longitude': -173.293, 'country': 'Albania'} 

user result:  {'ind': 7528, 'first_name': 'Abigail', 'last_name': 'Ali', 'age': 20, 'date_joined': datetime.datetime(2015, 10, 24, 11, 23, 51)} 

pin status code:  504 

pin status code:  504 

pin status code:  504 

**************************************************

pin result:  {'index': 2863, 'unique_id': '9

KeyboardInterrupt: 